In [96]:
!pip install numpy==1.23.5
!pip install scikit-surprise
!pip install lightfm

In [97]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

In [98]:
size = "small"

df_behaviors = pd.read_csv(f"MIND{size}_train/behaviors.tsv", sep="\t", names=['ImpressionID', 'UserID', 'Time', 'History', 'Impressions'])
df_news = pd.read_csv(f"MIND{size}_train/news.tsv", sep="\t", names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'TitleEntities', 'AbstractEntities'])

In [99]:
df_behaviors.head()

,ImpressionID,UserID,Time,History,Impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [100]:
df_news.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [101]:
df_behaviors["Time"] = pd.to_datetime(df_behaviors["Time"])
cutoff = pd.to_datetime("2019-11-14")

behavior_train = df_behaviors[df_behaviors["Time"] < cutoff].copy()
behavior_val   = df_behaviors[df_behaviors["Time"] >= cutoff].copy()

## Análisis de df_behaviors

In [102]:
print(f"Número de impresiones en train: {behavior_train.shape[0]}")
print(f"Número de impresiones en val: {behavior_val.shape[0]}")

Número de impresiones en train: 126695
Número de impresiones en val: 30270


In [103]:
print(f"Número de usuarios en train: {behavior_train['UserID'].nunique()}")
print(f"Número de usuarios en val: {behavior_val['UserID'].nunique()}")

Número de usuarios en train: 46012
Número de usuarios en val: 20179


In [104]:
usuarios_nuevos_por_impresion_train = behavior_train[behavior_train["History"].isnull() | (behavior_train["History"] == "")]
usuarios_nuevos_por_impresion_val = behavior_val[behavior_val["History"].isnull() | (behavior_val["History"] == "")]

In [105]:
print(f"Número de usuarios nuevos en train: {usuarios_nuevos_por_impresion_train['UserID'].nunique()}")
print(f"Número de usuarios nuevos en val: {usuarios_nuevos_por_impresion_val['UserID'].nunique()}")

Número de usuarios nuevos en train: 798
Número de usuarios nuevos en val: 476


In [106]:
impresiones_por_usuario_train = behavior_train["UserID"].value_counts()
impresiones_por_usuario_val = behavior_val["UserID"].value_counts()

In [107]:
print(f"Promedio de impresiones por usuario en train: {impresiones_por_usuario_train.mean():.2f}")
print(f"Promedio de impresiones por usuario en val  : {impresiones_por_usuario_val.mean():.2f}")

Promedio de impresiones por usuario en train: 2.75
Promedio de impresiones por usuario en val  : 1.50


In [108]:
df_behaviors["Time"] = pd.to_datetime(df_behaviors["Time"])
df_behaviors["Date"] = df_behaviors["Time"].dt.date
behavior_train["Date"] = behavior_train["Time"].dt.date
behavior_val["Date"] = behavior_val["Time"].dt.date

In [109]:
conteo_por_dia = df_behaviors["Date"].value_counts().sort_index()
conteo_por_dia_train = behavior_train["Date"].value_counts().sort_index()
conteo_por_dia_val = behavior_val["Date"].value_counts().sort_index()

In [110]:
promedio_impresiones_dia_train = conteo_por_dia_train.mean()
promedio_impresiones_dia_val = conteo_por_dia_val.mean()

print(f"Promedio de impresiones por día train: {promedio_impresiones_dia_train:.2f}")
print(f"Promedio de impresiones por día val  : {promedio_impresiones_dia_val:.2f}")

Promedio de impresiones por día train: 25339.00
Promedio de impresiones por día val  : 30270.00


In [111]:
behavior_train["HistoryLength"] = behavior_train["History"].fillna("").apply(lambda x: len(x.split()))
behavior_val["HistoryLength"] = behavior_val["History"].fillna("").apply(lambda x: len(x.split()))

In [112]:
print(f"Promedio de noticias en el historial por usuario: {behavior_train['HistoryLength'].mean():.2f}")
print(f"Promedio de noticias en el historial por usuario: {behavior_val['HistoryLength'].mean():.2f}")

Promedio de noticias en el historial por usuario: 32.41
Promedio de noticias en el historial por usuario: 33.09


In [113]:
num_interacciones = df_behaviors["Impressions"].apply(lambda x: len(x.split())).sum()

In [114]:
print(f"Número total de interacciones: {num_interacciones:,}")

Número total de interacciones: 5,843,444


### Creación de dataset típico de recomendación

In [115]:
interacciones = []

for _, row in df_behaviors.iterrows():
    user_id = row["UserID"]
    timestamp = row["Time"]
    impressions = row["Impressions"].split()

    for impression in impressions:
        news_id, clicked = impression.split('-')
        interacciones.append({
            "UserID": user_id,
            "NewsID": news_id,
            "Clicked": int(clicked),
            "Timestamp": timestamp
        })

df_interacciones = pd.DataFrame(interacciones)

In [116]:
df_interacciones.head()

,UserID,NewsID,Clicked,Timestamp
0,U13740,N55689,1,2019-11-11 09:05:58
1,U13740,N35729,0,2019-11-11 09:05:58
2,U91836,N20678,0,2019-11-12 18:11:30
3,U91836,N39317,0,2019-11-12 18:11:30
4,U91836,N58114,0,2019-11-12 18:11:30


In [117]:
cutoff = pd.to_datetime("2019-11-14")
train = df_interacciones[df_interacciones["Timestamp"] < cutoff]
val   = df_interacciones[df_interacciones["Timestamp"] >= cutoff]

In [118]:
print(f"Número de noticias mostradas train : {train['NewsID'].nunique()}")
print(f"Número de noticias mostradas val   : {val['NewsID'].nunique()}")
print(f"Número de noticias clickeadas train: {train[train['Clicked'] == 1]['NewsID'].nunique()}")
print(f"Número de noticias clickeadas val  : {val[val['Clicked'] == 1]['NewsID'].nunique()}")

Número de noticias mostradas train : 16978
Número de noticias mostradas val   : 6144
Número de noticias clickeadas train: 6398
Número de noticias clickeadas val  : 2097


In [119]:
n_usuarios_train = train["UserID"].nunique()
n_noticias_train = train["NewsID"].nunique()
n_interacciones_observadas_train = behavior_train.shape[0]

densidad_train = n_interacciones_observadas_train / (n_usuarios_train * n_noticias_train)

n_clics_train = train[train["Clicked"] == 1].shape[0]
densidad_clics_train = n_clics_train / (n_usuarios_train * n_noticias_train)

n_usuarios_val = val["UserID"].nunique()
n_noticias_val = val["NewsID"].nunique()
n_interacciones_observadas_val = behavior_val.shape[0]

densidad_val = n_interacciones_observadas_val / (n_usuarios_val * n_noticias_val)

n_clics_val = val[val["Clicked"] == 1].shape[0]
densidad_clics_val = n_clics_val / (n_usuarios_val * n_noticias_val)

In [120]:
print(f"Número de usuarios train: {n_usuarios_train}")
print(f"Número de usuarios val  : {n_usuarios_val}")

print(f"Número de noticias train: {n_noticias_train}")
print(f"Número de noticias val  : {n_noticias_val}")

print(f"Densidad (solo noticias mostradas) train : {densidad_train * 100:.3f}%")
print(f"Densidad (solo noticias mostradas) val   : {densidad_val * 100:.3f}%")

print(f"Densidad (solo noticias clickeadas) train: {densidad_clics_train * 100:.3f}%")
print(f"Densidad (solo noticias clickeadas) val  : {densidad_clics_val * 100:.3f}%")

Número de usuarios train: 46012
Número de usuarios val  : 20179
Número de noticias train: 16978
Número de noticias val  : 6144
Densidad (solo noticias mostradas) train : 0.016%
Densidad (solo noticias mostradas) val   : 0.024%
Densidad (solo noticias clickeadas) train: 0.024%
Densidad (solo noticias clickeadas) val  : 0.038%


In [121]:
noticias_clickeadas_por_usuario_train = train.groupby("UserID")["Clicked"].agg(["count", "sum"]).reset_index()
noticias_clickeadas_por_usuario_val = val.groupby("UserID")["Clicked"].agg(["count", "sum"]).reset_index()

In [122]:
noticias_clickeadas_por_usuario_train["porcentaje noticias vistas"] = noticias_clickeadas_por_usuario_train["sum"] / noticias_clickeadas_por_usuario_train["count"]
noticias_clickeadas_por_usuario_train["porcentaje noticias vistas"] = (noticias_clickeadas_por_usuario_train["porcentaje noticias vistas"] * 100)
noticias_clickeadas_por_usuario_val["porcentaje noticias vistas"] = noticias_clickeadas_por_usuario_val["sum"] / noticias_clickeadas_por_usuario_val["count"]
noticias_clickeadas_por_usuario_val["porcentaje noticias vistas"] = (noticias_clickeadas_por_usuario_val["porcentaje noticias vistas"] * 100)

In [123]:
promedio_mostradas_train = noticias_clickeadas_por_usuario_train["count"].mean()
promedio_mostradas_val = noticias_clickeadas_por_usuario_val["count"].mean()
print(f"Promedio de noticias mostradas por usuario train: {promedio_mostradas_train:.2f}")
print(f"Promedio de noticias mostradas por usuario val  : {promedio_mostradas_val:.2f}")

promedio_clickeadas_train = noticias_clickeadas_por_usuario_train["sum"].mean()
promedio_clickeadas_val = noticias_clickeadas_por_usuario_val["sum"].mean()
print(f"Promedio de noticias clickeadas por usuario train: {promedio_clickeadas_train:.2f}")
print(f"Promedio de noticias clickeadas por usuario val  : {promedio_clickeadas_val:.2f}")

promedio_clickeadas_porcentaje_train = noticias_clickeadas_por_usuario_train["porcentaje noticias vistas"].mean()
promedio_clickeadas_porcentaje_val = noticias_clickeadas_por_usuario_val["porcentaje noticias vistas"].mean()
print(f"Porcentaje promedio de noticias clickeadas por usuario train: {promedio_clickeadas_porcentaje_train:.2f}%")
print(f"Porcentaje promedio de noticias clickeadas por usuario val  : {promedio_clickeadas_porcentaje_val:.2f}%")

Promedio de noticias mostradas por usuario train: 100.43
Promedio de noticias mostradas por usuario val  : 60.58
Promedio de noticias clickeadas por usuario train: 4.12
Promedio de noticias clickeadas por usuario val  : 2.32
Porcentaje promedio de noticias clickeadas por usuario train: 9.07%
Porcentaje promedio de noticias clickeadas por usuario val  : 8.29%


In [124]:
clicks_por_noticia_train = train.groupby("NewsID")["Clicked"].agg(["count", "sum"]).reset_index()
clicks_por_noticia_val = val.groupby("NewsID")["Clicked"].agg(["count", "sum"]).reset_index()

In [125]:
clicks_por_noticia_train["porcentaje clicks noticia"] = clicks_por_noticia_train["sum"] / clicks_por_noticia_train["count"]
clicks_por_noticia_train["porcentaje clicks noticia"] = (clicks_por_noticia_train["porcentaje clicks noticia"] * 100)

clicks_por_noticia_val["porcentaje clicks noticia"] = clicks_por_noticia_val["sum"] / clicks_por_noticia_val["count"]
clicks_por_noticia_val["porcentaje clicks noticia"] = (clicks_por_noticia_val["porcentaje clicks noticia"] * 100)

In [126]:
promedio_mostradas_train = clicks_por_noticia_train["count"].mean()
promedio_mostradas_val = clicks_por_noticia_val["count"].mean()
print(f"Promedio de veces que se muestra cada noticia train: {promedio_mostradas_train:.2f}")
print(f"Promedio de veces que se muestra cada noticia val  : {promedio_mostradas_val:.2f}")

promedio_clickeadas_train = clicks_por_noticia_train["sum"].mean()
promedio_clickeadas_val = clicks_por_noticia_val["sum"].mean()
print(f"Promedio de clicks por noticia train: {promedio_clickeadas_train:.2f}")
print(f"Promedio de clicks por noticia val  : {promedio_clickeadas_val:.2f}")

promedio_clickeadas_porcentaje_train = clicks_por_noticia_train["porcentaje clicks noticia"].mean()
promedio_clickeadas_porcentaje_val = clicks_por_noticia_val["porcentaje clicks noticia"].mean()
print(f"Porcentaje promedio de clicks por noticia train: {promedio_clickeadas_porcentaje_train:.2f}%")
print(f"Porcentaje promedio de clicks por noticia val  : {promedio_clickeadas_porcentaje_val:.2f}%")

Promedio de veces que se muestra cada noticia train: 272.18
Promedio de veces que se muestra cada noticia val  : 198.96
Promedio de clicks por noticia train: 11.16
Promedio de clicks por noticia val  : 7.62
Porcentaje promedio de clicks por noticia train: 4.37%
Porcentaje promedio de clicks por noticia val  : 3.48%


## Análisis de df_news

In [127]:
df_news.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [128]:
print(f"Número de noticias: {df_news['NewsID'].nunique()}")
print(f"Número de categorías: {df_news['Category'].nunique()}")
print(f"Categorías: {df_news['Category'].unique().tolist()}")
print(f"Número de SubCategorías: {df_news['SubCategory'].nunique()}")

Número de noticias: 51282
Número de categorías: 17
Categorías: ['lifestyle', 'health', 'news', 'sports', 'weather', 'entertainment', 'autos', 'travel', 'foodanddrink', 'tv', 'finance', 'movies', 'video', 'music', 'kids', 'middleeast', 'northamerica']
Número de SubCategorías: 264


In [129]:
nulos_news = df_news.isnull().sum()[df_news.isnull().sum() > 0].reset_index()
nulos_news.columns = ["Columna", "Cantidad valores nulos"]
nulos_news

,Columna,Cantidad valores nulos
0,Abstract,2666
1,TitleEntities,3
2,AbstractEntities,4


In [130]:
df_news["Abstract"] = df_news["Abstract"].fillna("")
df_news["TitleEntities"] = df_news["TitleEntities"].fillna("[]")
df_news["AbstractEntities"] = df_news["AbstractEntities"].fillna("[]")

Se rellenan los valores debido a que es una cantidad importante de las noticias del dataset

In [131]:
df_news["AbstractEntities"][1]

'[{"Label": "Adipose tissue", "Type": "C", "WikidataId": "Q193583", "Confidence": 1.0, "OccurrenceOffsets": [97], "SurfaceForms": ["belly fat"]}]'

## Recomendadores

In [132]:
df_interacciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5843444 entries, 0 to 5843443
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   UserID     object        
 1   NewsID     object        
 2   Clicked    int64         
 3   Timestamp  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 178.3+ MB


In [133]:
print(f"El dataset de entrenamiento tiene un {(train.shape[0] / df_interacciones.shape[0]) * 100:.2f}% de los datos")
print(f"El dataset de validación tiene un {(val.shape[0] / df_interacciones.shape[0]) * 100:.2f}% de los datos")

El dataset de entrenamiento tiene un 79.08% de los datos
El dataset de validación tiene un 20.92% de los datos


In [134]:
noticias_disponibles = df_interacciones["NewsID"].unique()

In [135]:
from collections import defaultdict

clicks_verdaderos = defaultdict(set)

for _, row in val[val["Clicked"] == 1].iterrows():
    clicks_verdaderos[row["UserID"]].add(row["NewsID"])

In [136]:
def recomendar_random(noticias, k=10):
    return np.random.choice(noticias, size=k, replace=False).tolist()

recomendaciones_random = {
    user: recomendar_random(noticias_disponibles, k=10)
    for user in clicks_verdaderos.keys()
}

In [137]:
recall_sum = 0

for user, reales in clicks_verdaderos.items():
    predichas = set(recomendaciones_random[user])
    hits = len(predichas & reales)
    recall_sum += hits / len(reales)

recall_at_10 = recall_sum / len(clicks_verdaderos)
print(f"Recall@10 del recomendador aleatorio: {recall_at_10:.4f}")

Recall@10 del recomendador aleatorio: 0.0004


### Most Popular

In [138]:
popularidad = (train[train["Clicked"] == 1].groupby("NewsID").size().sort_values(ascending=False)
)

top_noticias = popularidad.index.tolist()  # Noticias ordenadas de más a menos populares

In [139]:
recomendaciones_populares = {
    user: top_noticias[:10]
    for user in clicks_verdaderos.keys()  # los mismos usuarios con ground truth que usaste antes
}

In [140]:
recall_sum = 0

for user, reales in clicks_verdaderos.items():
    predichas = set(recomendaciones_populares[user])
    hits = len(predichas & reales)
    recall_sum += hits / len(reales)

recall_at_10_popular = recall_sum / len(clicks_verdaderos)
print(f"Recall@10 del recomendador por popularidad: {recall_at_10_popular:.4f}")


Recall@10 del recomendador por popularidad: 0.0001


In [141]:
# Crear diccionario de popularidad
popularidad_dict = popularidad.to_dict()

# Recomendaciones populares entre las noticias que realmente se le mostraron a cada usuario
recomendaciones_populares_filtradas = {}

# Agrupar validación por usuario y timestamp
for (user, timestamp), group in train.groupby(["UserID", "Timestamp"]):
    noticias_mostradas = group["NewsID"].tolist()

    # Ordenar las noticias por popularidad (entre las mostradas)
    ordenadas = sorted(noticias_mostradas, key=lambda x: popularidad_dict.get(x, 0), reverse=True)

    recomendaciones_populares_filtradas[user] = ordenadas[:10]  # top-10 entre sus opciones

In [142]:
recall_sum = 0

for user, reales in clicks_verdaderos.items():
    predichas = set(recomendaciones_populares_filtradas.get(user, []))
    hits = len(predichas & reales)
    recall_sum += hits / len(reales)

recall_at_10_pop_filtrado = recall_sum / len(clicks_verdaderos)
print(f"Recall@10 del recomendador por popularidad (filtrado): {recall_at_10_pop_filtrado:.4f}")


Recall@10 del recomendador por popularidad (filtrado): 0.0045


In [143]:
def evaluar_recommendaciones_at_k(recomendaciones, clicks_verdaderos, k=10):
    recall_sum = 0
    precision_sum = 0
    hits_sum = 0
    ndcg_sum = 0
    map_sum = 0
    usuarios_con_clics = 0

    for user, reales in clicks_verdaderos.items():
        if not reales:
            continue

        predichas = recomendaciones.get(user, [])[:k]
        if not predichas:
            continue

        predichas_set = set(predichas)
        hits = len(predichas_set & reales)

        recall = hits / len(reales)
        precision = hits / k
        hitrate = 1 if hits > 0 else 0

        # nDCG@K
        dcg = 0
        for i, item in enumerate(predichas):
            if item in reales:
                dcg += 1 / np.log2(i + 2)
        idcg = sum(1 / np.log2(i + 2) for i in range(min(len(reales), k)))
        ndcg = dcg / idcg if idcg > 0 else 0

        # MAP@K
        hits_acumulados = 0
        precisiones = []
        for i, item in enumerate(predichas):
            if item in reales:
                hits_acumulados += 1
                precisiones.append(hits_acumulados / (i + 1))
        ap = np.mean(precisiones) if precisiones else 0

        recall_sum += recall
        precision_sum += precision
        hits_sum += hitrate
        ndcg_sum += ndcg
        map_sum += ap
        usuarios_con_clics += 1

    if usuarios_con_clics == 0:
        return {m: 0.0 for m in ["Recall@K", "nDCG@K", "MAP@K"]}

    return {
        "Recall@K": recall_sum / usuarios_con_clics,
        "MAP@K": map_sum / usuarios_con_clics,
        "nDCG@K": ndcg_sum / usuarios_con_clics
    }


In [144]:
def evaluar_diversity_novelty(recomendaciones, frecuencia_items, id_a_categoria=None, k=10):
    diversity_total = 0
    novelty_total = 0
    usuarios_validos = 0

    for user, items in recomendaciones.items():
        items = items[:k]
        if len(items) < 2:
            continue

        # Novelty
        novelty = 0
        for item in items:
            p = frecuencia_items.get(item, 1e-6)
            novelty += -np.log2(p)
        novelty /= len(items)

        # Diversity (categorías diferentes)
        if id_a_categoria:
            categorias = [id_a_categoria.get(i, None) for i in items]
            diferentes = 0
            total_pares = 0
            for i in range(len(categorias)):
                for j in range(i+1, len(categorias)):
                    if categorias[i] is not None and categorias[j] is not None:
                        total_pares += 1
                        if categorias[i] != categorias[j]:
                            diferentes += 1
            diversity = diferentes / total_pares if total_pares > 0 else 0
        else:
            diversity = 0

        diversity_total += diversity
        novelty_total += novelty
        usuarios_validos += 1

    return {
        "Novelty@K": novelty_total / usuarios_validos,
        "Diversity@K": diversity_total / usuarios_validos
    } if usuarios_validos > 0 else {
        "Novelty@K": 0.0,
        "Diversity@K": 0.0
    }

In [145]:
def imprimir_metricas_completas(recomendaciones, clicks_verdaderos, frecuencia_items, id_a_categoria, k=10):
    resultados_1 = evaluar_recommendaciones_at_k(recomendaciones, clicks_verdaderos, k)
    resultados_2 = evaluar_diversity_novelty(recomendaciones, frecuencia_items, id_a_categoria, k)
    resultados = {**resultados_1, **resultados_2}
    for metrica, valor in resultados.items():
        print(f"{metrica}: {valor:.4f}")

In [146]:
# Requiere: diccionario con popularidad normalizada
frecuencia_items = train["NewsID"].value_counts(normalize=True).to_dict()

# Opcional para Diversity: categoría de cada noticia
id_a_categoria = df_news.set_index("NewsID")["Category"].to_dict()

In [147]:
print("RecSys Random")
imprimir_metricas_completas(recomendaciones_random, clicks_verdaderos, frecuencia_items, id_a_categoria, k=10)

print("\nRecSys MostPopular")
imprimir_metricas_completas(recomendaciones_populares, clicks_verdaderos, frecuencia_items, id_a_categoria, k=10)

print("\nRecSys MostPopular filtered")
imprimir_metricas_completas(recomendaciones_populares_filtradas, clicks_verdaderos, frecuencia_items, id_a_categoria, k=10)

RecSys Random
Recall@K: 0.0004
MAP@K: 0.0004
nDCG@K: 0.0002
Novelty@K: 18.7992
Diversity@K: 0.8183

RecSys MostPopular
Recall@K: 0.0001
MAP@K: 0.0000
nDCG@K: 0.0000
Novelty@K: 8.5706
Diversity@K: 0.8222

RecSys MostPopular filtered
Recall@K: 0.0056
MAP@K: 0.0031
nDCG@K: 0.0030
Novelty@K: 9.0687
Diversity@K: 0.8396
